In [3]:
import os
import boto3
from chalice import Chalice, AuthResponse, Rate
from chalicelib import auth, db

# following is from the python notebook
import pandas as pd
from apify_client import ApifyClient
from datetime import datetime
from opensearchpy import OpenSearch, helpers

app = Chalice(app_name='mytodo')
app.debug = True
_DB = None
_USER_DB = None
apify_client = ApifyClient('apify_api_IY3hoxDRTo2d2iF7NoZXPIOtFlDb8J1cSkPQ') 

In [2]:
!pip install pandas

In [4]:
#@app.schedule(Rate(24, unit=Rate.HOURS))
#def every_day(event):# this will be attached to lambda function
@app.schedule(Rate(24, unit=Rate.HOURS))
def every_24_hour(event): # event
    print('updated 2 !!!!!!')
    # Start an actor and wait for it to finish
    position_df = pd.DataFrame()
    # job_titles_old = ["Data Analyst", "Data Engineer", "Data Scientist", "Software Developer", "Product Manager", "Digital Marketer"]
    job_titles = [ "Software Engineer",  "Data Engineer", "Data Scientist"] #  
    locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver", "w+CAIQICIITW9udHJlYWw=": "Montreal"}

    for uule in locations:
        print(locations[uule])
        for jt in job_titles:
            print(jt)
            run_input = {
                "csvFriendlyOutput": True,
                "includeUnfilteredResults": False,
                "maxConcurrency": 10,
                "maxPagesPerQuery": 2,
                "queries": f"https://www.google.com/search?ibp=htl;jobs&q={jt}&uule={uule}",
                "saveHtml": False,
                "saveHtmlToKeyValueStore": False,
            }

            actor_call = apify_client.actor(
                'dan.scraper/google-jobs-scraper').call(run_input=run_input)

            dataset_items = apify_client.dataset(
                actor_call['defaultDatasetId']).list_items().items

            d = pd.DataFrame(dataset_items)
            d["query"] = jt
            d["location"] = locations[uule]
            d["run_time"] = str(datetime.now())

            position_df = pd.concat([position_df, d])

        print("="*30)

    position_df.drop(columns='thumbnail', inplace=True)

    host = 'search-swift-hire-dev-jfmldmym4cfbiwdhwmtuqq6ihy.us-west-2.es.amazonaws.com'
    port = 443
    auth = ('swift', 'Hire123!') # For testing only. Don't store credentials in code.

    print('connect to OpenSearch client: ')
    client = OpenSearch(
        hosts = [{'host': host, 'port': port}],
        http_compress = True, # enables gzip compression for request bodies
        http_auth = auth,
        use_ssl = True,
        ssl_assert_hostname = False,
        ssl_show_warn = False,
    )
    print('index swift_dev_felix_kelly ')
    index_name = "swift_dev_felix_kelly"

    if not client.indices.exists(index_name):
        client.indices.create(index=index_name)
    def doc_generator(df):
        for i, row in df.iterrows():
            doc = {
                "_index": index_name,
                "_source": row.to_dict(),
            }
            yield doc
    helpers.bulk(client, doc_generator(position_df))

    print("Data Saved to ES. Done.") 

In [12]:
#every_24_hour()

updated 2 !!!!!!
Toronto
Software Engineer
Data Engineer
Data Scientist
Vancourver
Software Engineer
Data Engineer
Data Scientist
Montreal
Software Engineer
Data Engineer
Data Scientist
connect to OpenSearch client: 
index swift_dev_felix_kelly 
Data Saved to ES. Done.


In [6]:
print("end")

end
